### 앙상블 종류
- 보팅 : 여러 개의 다른 모델로 예측한 결과를 투표/평균을 활용하여 최종 결정
    - 하드 보팅: 다수결
    - 소프트 보팅 : 해당 클래스
- 배깅 : 여러 개의 같은 모델로 예측한 결과를 투표/평균을 활용하여 최종결졍
    - 대표적인 배깅 : RandomForest(랜덤포레스트)
- 부스팅 : 여러 개의 같은 모델이 순차적으로 학습-에측하여, 가중치를 부여해 오류를 개선하는 방식


In [1]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [ ]:
log_clf = LogisticRegression(max_iter=10000)
# 주어진 데이터가 특정 클래스에 속할 확률을 추정
# max_iter : 알고리즘이 수렴할 때까지 반복하여 최대 학습 횟수를 지정

svm_clf = SVC(probability=True)
# 클래스를 잘 구분할 수 있는 최적의 결정경계를 찾는 알고리즘
# probability : SVM 원래 확률값을 제공하진 않음, True로 설정해서 확률값을 받아보겠다.

In [ ]:
hard_bot_clf = VotingClassifier(
    estimators=[("log", log_clf),("svm",svm_clf)], #로지스틱회귀와 svm을 결합
    voting="hard" #voting방식 지정
)

In [ ]:
# 모델학습
hard_bot_clf.fit(x_trian,y_train)

In [ ]:
soft_bot_clf = VotingClassifier(
    estimators=[("log", log_clf),("svm",svm_clf)], #로지스틱회귀와 svm을 결합
    voting="soft" #voting방식 지정
)

In [ ]:
soft_bot_clf.fit(x_trian,y_train)

In [ ]:
cross_val_score(soft_bot_clf,hard_bot_clf)

## 배깅 - 랜덤포레스트
- 여러개의 같은 모델이 예측한 결과를 투표/평균으로 최종 예측
- 대표적으로 랜덤포레스트(Random Forest)
- 그리드서치(Gridsearch)

In [ ]:
#모델 불러오기
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier(
    n_estimators=100,# 100개의 트리
    random_state=2024 # 랜덤포레스트 특성 상 난수 고정이필요.
)

In [ ]:
#모델 학습
rf_clf.fit(x_train,y_train)

In [ ]:
cross_val_score(rf_clf,x_train,y_train,cv=5)

## GridsearchCV 적용해서 최적의 하이퍼 파라미터 찾기

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {
    "n_estimators" : [50,100,200], #트리모델 개수 설정
    "max_depth" : [3,5,7], # 트리의 깊이 설정
    "max_leaf_nodes" : [6,12,24], #리프토드의 최대 개수 제한
    "min_samples_split" :[3,5,7] #노드를 분할하기 위한 최소 샘플 수 지정
}

# 2. 대상 모델 설정
rf_clf = RandomForestClassifier(random_state=2024)

# 그리드 서치 설정
grid = GridSearchCV(rf_clf, # 모델설정
                    params, #하이퍼파라미터 경우의 수
                    cv=5, #교차검증 군집수 설정
                    scoring="accurary", # 평가지표 설정
                    n_jobs= -1 #사용 가능한 모든 CPU 코어를 사용하여 병렬처리를 수행
)

# 실행 -> 조합을 찾기위한 fit
grid.fit(x_train,y_train)

In [ ]:
#최적의 하이퍼파라미터 조합 확인
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
besf_rf_clf = RandomForestClassifier(
    max_depth=5,
    max_leaf_nodes=24,
    min_samples_split=7,
    n_estimators=100
)

In [ ]:
# 검증으로 성능확인
cross_val_score(besf_rf_clf,x_trian,y_train,cv=5)

In [2]:
#특성 중요도
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
#특성 중요도
fi = besf_rf_clf.feature_importances_
fi_series = pd.Series(fi,index=X_train.columns)
fi_top20 = fi_series.sort_values(ascending=False)

In [ ]:
sns.barplot(x=fi_top20,y=fi_top20.index)
plt.show()

#### Boosting
- 여러 개의 같은 종류의 모델이 순차적으로 학습-예측하며 가중치를 부여해 오류를 개성하는 방식

### 부스팅 모델 종류
- AdaBoost(Adaptive Boosting)
    : 오분류된 데이터 포인트에 더 큰 가중치를 할당하고, 이를 통해 새로운 모델이 이전의 오류에 적응해서 개선하는 방식.
    : 각 반복에서 올바르게 분류된 샘플(정분류)은 가중치를 감소하고, 잘못분류된 샘플(오분류)의 가중치는 증가하며,
        오분류된 샘플을 보완해나가는 방식
    
- GBM(Gradient Boosting)
    : 손실 함수의 기울기를 사용하여 연속적으로 모델을 개선
    : 유연성이 높음

- XGBoost
    : GBM을 확장하고 성능과 속도를 최적화
    : 대규모 데이터셋과 복잡한 문제에 효과적(Kaggle 대회에서도 많이 사용된 모델// 딥러닝이 나오기전까지)
    
- Light GBM
...


#### 분류 평가 지표
- accuracy(정확도) : (TP+TN) /(TP+TN+FP+FN)
    : 전체 중에 맞춘(예측에 성공한) 비율
    : 일반적으로 많이 쓰임

- precision(정밀도) : TP / (TP+FP)
    : 양성으로 예측된 샘플중 실제 양성인 샘플의 비율
    : 잘못 분류되는 것을 최소화 할떄 가장 중요
    : 정밀도가 높다는 것은 모델이 "양성"이라고 분류한 케이스들이 "실제 양성"일 확률이 높다는 것을 의미
    -> 모델이 얼마나 적은 수의 "거짓 양성"오류를 만들어내는지 확인하는 지표
    - 스팸 메일 분류 : 스팸으로 잘못분류된 정상이메일(FP)이 중요한 메일일 수도 있으므로, 정상 이메일이
                      스팸으로 분류되는 경우를 최소화 하려면 정밀도를 높이는 것이 중요.

- recall(재현율) : TP/(TP+FN)
    : 실제 양성 샘플 중 양성올 올바르게 예측된 샘플의 비율
    : 실제 양성을 높지지 않는 것이 중요할 때 선호됨
    : 재현율이 높다는 것은 "실제 양성"인 케이스를 모델이 놓치지 않고 잘 포착한다는 것을 의미.
    : 암진단 : 실제 실병이 있는 환자를 질병이 없다고 잘못 분류하지 않는 것이 중요할때 재현율이 중요.

- F1 score : (2*정밀도*재현율) / (정밀도+재현율)
    : 정밀도와 재현율의 조화평균
    : 두 지표가 한 쪽에 치우치지 않고, 양성케이스를 정밀하고 효율적으로 식별하는 정도를 수치화


In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score